# Restoring the Scheduler

### 1. Read the queue

In [46]:
with open('res/dialup-log-1985.txt', 'r') as f:
    data = f.readlines()

# Déterminons l´index où l´on trouve 'Synchronised.'
# On ne garde que les lignes après cette ligne
start_index = 0
for i, line in enumerate(data):
    if 'Synchronised.' in line:
        start_index = i + 1
        break
data = data[start_index:]
print(f"Nombre de lignes après 'Synchronised.': {len(data)}")

# Filtrons les lignes ne contenant que 'RECV MSG: 0x'
messages_recus = list(filter(lambda line: all(['RECV MSG: 0x' in line, 'duplicate' not in line, 'resend' not in line]), data))

Nombre de lignes après 'Synchronised.': 41


### 2. Decode the headers

In [ ]:
from datetime import datetime


class Message:

    counter = 1

    def __init__(self, message: str) -> None:
        self._message = message
        self.date, self.header = self.decrypte_hex_line(message)
        self.binary_header = format(int(self.header, 16), '032b')
        self.index = Message.counter
        Message.counter += 1

    def decrypte_hex_line(self, line: str) -> tuple[datetime, str]:
        date, _, _, hex_header = tuple(filter(None, line.strip().split(' ')))
        return datetime.strptime(date, '%H:%M:%S'), hex_header
    
    @property
    def priority(self) -> str:
        return self.binary_header[-3:]
    
    @property
    def timestamp(self) -> str:
        return self.binary_header[-16:-3]
    
    @property
    def aging_factor(self) -> str:
        return self.binary_header[-20:-16]
    
    @property
    def payload(self) -> str:
        return self.binary_header[:-20]
    
    @property
    def priority_value(self) -> int:
        return int(self.priority, 2)

    @property
    def timestamp_value(self) -> int:
        return int(self.timestamp, 2)

    @property
    def aging_factor_value(self) -> int:
        return int(self.aging_factor, 2)
    
    @property
    def message(self) -> str:
        return self._message
    
    @message.setter
    def message(self, new_hex_message: str) -> None:
        self._message = new_hex_message
        self.binary_header = format(int(new_hex_message, 16), '032b')

    def __lt__(self, other: 'Message') -> bool:
        if not isinstance(other, Message):
            return NotImplemented
        # Comparateur: priorité (asc), timestamp (asc), aging factor (desc), puis date pour stabilité
        if self.priority_value != other.priority_value:
            return self.priority_value < other.priority_value
        if self.timestamp_value != other.timestamp_value:
            return self.timestamp_value < other.timestamp_value
        if self.aging_factor_value != other.aging_factor_value:
            return self.aging_factor_value > other.aging_factor_value
        return self.date < other.date
    

def exclus_timestamp_1fff(message: Message):
    def convert_32bit_to_hex(bits) -> str:
        return hex(int(bits, 2))
    
    if convert_32bit_to_hex(message.timestamp) != '0x1fff':
        return message
    return None
    
    
corrupted_messages = [Message(line) for line in messages_recus]
# Excluons les messages avec un timestamp 0x1FFF
corrupted_messages = list(filter(None, map(exclus_timestamp_1fff, corrupted_messages)))
print(f"Nombre de messages corrompus après exclusion des timestamps 0x1FFF: {len(corrupted_messages)}")

Nombre de messages corrompus après exclusion des timestamps 0x1FFF: 28


### 3. Repair corrupted headers

In [48]:
# Isolons les 20 premiers bits des messages corrompus

def repair_corrupted_headers(message: Message) -> Message:
    # Seuls les messages 25 à 28 (ordre d'arrivée) sont corrompus
    if 25 <= message.index <= 28:
        # Récupère la valeur entière du header sur 32 bits
        header_int = int(message.header, 16)
        lower_20 = header_int & 0xFFFFF  # bits 0-19

        def rotation_droite_20b(value: int, n: int) -> int:
            """Rotation circulaire à droite sur 20 bits"""
            return ((value >> n) | (value << (20 - n))) & 0xFFFFF

        rotated_20 = rotation_droite_20b(lower_20, 2)  # inverse la rotation gauche de 2
        repaired_int = (header_int & ~0xFFFFF) | rotated_20
        repaired_header_hex = format(repaired_int, '08x')

        # Mise à jour du message (recalcule binary_header)
        message.message = repaired_header_hex

    return message


repaired_messages = list(map(repair_corrupted_headers, corrupted_messages))
print("2 premiers messages réparés:")
print(repaired_messages[0].message)
print(repaired_messages[1].message)

2 premiers messages réparés:
09:16:25 RECV MSG: 0x84960A5E

09:16:29 RECV MSG: 0xF4064DA7



### 4. Sort the queue

In [49]:
# Tri des messages réparés selon priorité asc, timestamp asc, aging factor desc
sorted_messages = sorted(repaired_messages)
print("3 premiers messages triés (header hex):")
for msg in sorted_messages[:3]:
    print(msg.message)


3 premiers messages triés (header hex):
3fb106d0
09:16:57 RECV MSG: 0xCC3017E0

09:17:01 RECV MSG: 0x7CE75548



### 5. Produce the key

In [51]:
# 5. Produce the key
# Les messages triés sont dans sorted_messages et portent leur index d'arrivée (01..32)
original_indices = [f"{msg.index}" for msg in sorted_messages]
key = "-".join(original_indices)
print(f"Clé reconstituée ({len(original_indices)} messages): {key}")

Clé reconstituée (28 messages): 25-9-10-13-15-14-23-20-5-18-26-12-28-16-4-21-22-27-11-17-7-3-6-1-19-24-2-8
